# Data Analytics for Bank Dataset

### Table of Contents
1.  [Data Loading and Initial Exploration](#data-loading-and-initial-exploration)
2.  [Data Cleaning](#data-cleaning)
3.  [Handling Duplicates](#handling-duplicates)
4.  [Target Variable Distribution](#target-variable-distribution)
5.  [Distribution of Numerical Features](#distribution-of-numerical-features)
6.  [Distribution of Categorical Features](#distribution-of-categorical-features)
7.  [Numerical Feature Distribution by Class](#numerical-feature-distribution-by-class)
8.  [Interaction between Categorical Features](#interaction-between-categorical-features)
9.  [Correlation Matrix of Numerical Features](#correlation-matrix-of-numerical-features)
10. [Mutual Information Scores](#mutual-information-scores)
11. [Feature Importance with Random Forest](#feature-importance-with-random-forest)
12. [t-SNE Visualization](#t-sne-visualization)
13. [UMAP Visualization](#umap-visualization)
14. [Subgroup Discovery](#subgroup-discovery)
15. [Pair Plot Analysis](#pair-plot-analysis)

We first start with loading the dataset and performing some basic data exploration and visualization. The dataset contains various features related to bank customers, including demographic information, account details, and whether they subscribed to a term deposit.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load the dataset
bank_data = pd.read_csv(os.path.join('..', 'data', 'real', 'bank.csv'))
column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'Class']
bank_data.columns = column_names
bank_data

Looking at the dataset, we can see, that instead of strings, some of the categorical features are stored as byte-string literals (e.g., "b'married'"). We will need to clean these up before proceeding with our analysis.
With the dataset loaded, we can now proceed to explore its structure, check for missing values, and visualize the data to understand the distribution of features and the target variable. The describe() method will give us a summary of the numerical features, while the info() method will provide insights into the data types and non-null counts of each column.

In [ ]:
bank_data.describe()

Looking at the info() method output, we can see that the dataset contains a mix of numerical and categorical features. The 'Class' column is our target variable, indicating whether a customer subscribed to a term deposit. Interestingly the dataset has no missing values, which is a good sign for our analysis, as we don't think of how to handle missing data.

In [ ]:
bank_data.info()

Next, we will check for any byte-string literals in the categorical columns and convert them to regular strings. This is necessary because some of the categorical features are stored as byte-string literals (e.g., "b'married'"), which can complicate our analysis, vizualization and interpretation of the data.

In [ ]:
for col in bank_data.columns:
    if bank_data[col].dtype == 'object':
        # Drop NA to safely access an element for the check
        non_null_series = bank_data[col].dropna()
        if not non_null_series.empty:
            first_entry = non_null_series.iloc[0]
            # Check if the string looks like a byte-string literal, e.g., "b'married'"
            if isinstance(first_entry, str) and first_entry.startswith("b'") and first_entry.endswith("'"):
                # Use vectorized .str.slice() to remove the first 2 and the last character
                bank_data[col] = bank_data[col].str.slice(2, -1)

bank_data

Now that we have cleaned the categorical columns, we can proceed with our data exploration and visualization. We will start by checking for duplicates in the dataset, which can skew our analysis if present.

In [ ]:
# Check for duplicates in the dataset
duplicates = bank_data.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")
# If duplicates are found, we can remove them
if duplicates > 0:
    bank_data = bank_data.drop_duplicates()
    print(f"Duplicates removed. New number of rows: {len(bank_data)}")
else:
    print("No duplicates found.")

Now we will start by checking the distribution of the target variable 'Class'. We can see, that the dataset is imbalanced, with a significantly higher number of customers who did not subscribe to a term deposit compared to those who did. This is important to consider when building predictive models, as it may affect the model's performance and evaluation metrics.

In [ ]:
# Show distribution of target variable 'Class'
sns.countplot(x='Class', data=bank_data)

We now proceed to visualize the distribution of numerical features in the dataset. We therefore plot histograms for each numerical feature to understand their distributions. We can observe, that age and day are having more variation, while balance, duration, pdays, previous and campaign have more concentrated distributions.

In [ ]:
# Select numerical columns
numerical_cols = bank_data.select_dtypes(include=np.number).columns

# Plot histograms
bank_data[numerical_cols].hist(figsize=(12, 10), bins=20, xlabelsize=8, ylabelsize=8)
plt.suptitle('Histograms of Numerical Features', x=0.5, y=1.02, fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

Next, we will visualize the distribution of categorical features in the dataset. We will create count plots for each categorical variable to understand their distributions and how they relate to the target variable 'Class'. This will help us identify any patterns or relationships between the features and the target variable.

For most features we observe, that even though we have a lot of different categories, the relative distribution is quite similar for both classes. However, we can see that the 'job' and 'education' features have some categories with a higher proportion of customers who subscribed to a term deposit.

In [ ]:
# Select categorical columns (excluding the target 'Class')
categorical_cols = bank_data.select_dtypes(include=['object']).columns

# Determine grid size
n_cols = 3
n_rows = 4

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10 * n_cols, 5 * n_rows))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Create count plots for each categorical variable against the target
for i, col in enumerate(categorical_cols):
    ax = axes[i]
    sns.countplot(x=col, hue='Class', data=bank_data, order=bank_data[col].value_counts().index, ax=ax)
    ax.set_title(f'{col}')
    ax.tick_params(axis='x', rotation=45)
    ax.set_xlabel(col) # Set x-axis label for clarity

# Hide any unused subplots
for j in range(len(categorical_cols), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Distribution of Categorical Features by Class', fontsize=16, y=1.0)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.show()

For the numerical features, we also want to take a look at the distribution of each numerical variable against the target variable 'Class'. We will create box plots for each numerical feature to visualize how the distributions differ between the two classes. This will help us identify any significant differences in the distributions of numerical features based on whether a customer subscribed to a term deposit or not.

Here again we can see, that the distributions of most numerical features are quite similar for both classes, but there are some features like 'balance', 'duration', and 'pdays' that show more distinct differences between the two classes. This suggests that these features may be more informative for predicting whether a customer will subscribe to a term deposit.

In [ ]:
# Determine grid size for the plots
n_cols = 3
n_rows = 3

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(8 * n_cols, 5 * n_rows))
axes = axes.flatten()  # Flatten the axes array for easy iteration

# Create box plots for each numerical variable against the target
for i, col in enumerate(numerical_cols):
    ax = axes[i]
    sns.boxplot(x='Class', y=col, data=bank_data, ax=ax)
    ax.set_title(f'{col}')
    ax.set_xlabel('Class')
    ax.set_ylabel(col)

# Hide any unused subplots
for j in range(len(numerical_cols), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle('Numerical Feature Distribution by Class', fontsize=16, y=1.0)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.show()

Next we feature a faceted count plot that visualizes the interaction between two categorical features — job and marital status — on the target Class (subscription outcome).

1. The Strongest Signal: The "Retired" Segment is Highly Successful
- Observation: Look at the job = retired panel (second row, second plot). The orange bars (Class 2) are proportionally much larger compared to their blue counterparts than in any other job category.
- Interpretation: Retired individuals have a significantly higher subscription rate across all marital statuses. This is a prime target demographic. The bank's product or marketing approach seems to resonate very strongly with this group.
2. The Weakest Signal: "Entrepreneurs" and "Blue-Collar" Workers Have Low Success Rates
- Observation: In the job = entrepreneur panel, the orange bars are almost invisible. In the job = blue-collar panel, the blue bars are overwhelmingly dominant, meaning the ratio of orange to blue is very small.
- Interpretation: These two job segments have a very low propensity to subscribe. Targeting them with the current campaign strategy would likely be inefficient and have a low return on investment.
3. The Influence of Marital Status Varies by Job
This is where the interaction becomes interesting:

For most jobs (management, technician, admin), the subscription rate for single individuals appears to be slightly higher than for married individuals. Even though there are more married people in total, the ratio of orange-to-blue is often better for the single category.
An exception is the retired group, where both married and divorced individuals have very high success rates, possibly even higher than single retirees.
4. Identifying Promising Target Niches
By combining these observations, we can identify specific high-value customer profiles:

- Top Tier Target: Any retired person.
- Strong Secondary Targets:
    - A single person in management.
    - A single person in admin.
    - A divorced person who is retired.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.catplot(x="marital", hue="Class", col="job",
                data=bank_data, kind="count",
                height=4, aspect=.7, col_wrap=4,
                col_order=bank_data['job'].value_counts().index[:8])

g.fig.suptitle('Subscription by Job and Marital Status', y=1.03)

g.fig.subplots_adjust(hspace=0.5) 

for ax in g.axes.flat:
    ax.set_xlabel("Marital Status")
    
    ax.tick_params(labelbottom=True)
    labels = ax.get_xticklabels()
    ax.set_xticklabels(labels, rotation=45, ha='right')

plt.show()

We now look at the correlation between numerical features in the dataset. We will create a correlation matrix to visualize the relationships between numerical features. This will help us identify any strong correlations that may exist between features, which can be useful for feature selection and understanding the data.

Most features have very weak or no linear correlation with each other. Most of the values are very close to 0.00. This is important because it suggests that simple linear models might struggle to find strong patterns using these features alone.

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(bank_data[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix of Numerical Features')
plt.show()

We also want to visualize the relationship between categorical features and the target variable 'Class'. Therefor we will use mutual information to quantify the relationship between the features and the target variable. Mutual information measures the amount of information gained about one variable through another variable. We will calculate mutual information scores for each feature with respect to the target variable 'Class'. Another advantage of mutual information is that it can capture non-linear relationships, which is particularly useful as we have seen that most features have very weak or no linear correlation with each other.

We can observe that features like 'duration', 'poutcome', and 'pdays' have relatively high mutual information scores, indicating that they are more informative for predicting whether a customer will subscribe to a term deposit. This indicates, that the duration of the last contact, the outcome of the previous marketing campaign, and the number of days since the last contact are important features for predicting customer behavior in this context.


In [ ]:
# Calculate mutual information for both numerical and categorical features
from sklearn.feature_selection import mutual_info_classif
# Prepare the data for mutual information calculation
X = bank_data.drop('Class', axis=1)
y = bank_data['Class']
# One-hot encode categorical features
X_encoded = pd.get_dummies(X, drop_first=True)
# Calculate mutual information
mi_scores = mutual_info_classif(X_encoded, y, discrete_features='auto', random_state=42)
# Create a DataFrame for mutual information scores
mi_df = pd.DataFrame({'Feature': X_encoded.columns, 'Mutual Information': mi_scores})
# Sort by mutual information scores
mi_df = mi_df.sort_values(by='Mutual Information', ascending=False)
# Plot mutual information scores
plt.figure(figsize=(10, 6))
sns.barplot(x='Mutual Information', y='Feature', data=mi_df)
plt.title('Mutual Information Scores of Features')
plt.xlabel('Mutual Information Score')
plt.ylabel('Features')
plt.show()

Now we will apply a Random Forest model to the dataset to assess feature importance. Random Forest is a powerful ensemble learning method that can handle both numerical and categorical features, and it provides a straightforward way to evaluate feature importance based on how much each feature contributes to the model's predictions. We use it specifically for data exploration and feature selection, rather than for building a predictive model. Therefore, we will not perform any hyperparameter tuning or cross-validation, as our goal is to understand the importance of features in the context of this dataset. We want to compare these feature importances with the mutual information scores we calculated earlier.

We observe that the feature importances from the Random Forest model partly align with the mutual information scores we calculated earlier. Duration has the highest importance score. Followed by balance, age and day, which is different from the mutual information scores, where balance was important, but not under the top three and age was only in the middle of mutual information values. Especially the importance of 'pdays' and 'poutcome' is much lower in the Random Forest model compared to the mutual information scores, while day and campaign are more important in the Random Forest model.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Prepare data for Random Forest model
# One-hot encode categorical features
bank_data_encoded = pd.get_dummies(bank_data.drop('Class', axis=1), drop_first=True)
# We can reuse the one-hot encoded data from the previous step
# X is your feature matrix (bank_data_encoded), y is the target variable
X = bank_data_encoded
y = bank_data['Class']

# Initialize and train a Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X, y)

# Get feature importances from the trained model
importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame for easier visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 12))
sns.barplot(x='importance', y='feature', data=feature_importance_df)
plt.title('Feature Importance from Random Forest')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.show()

Now we will apply t-SNE (t-distributed Stochastic Neighbor Embedding) to visualize the high-dimensional data in a lower-dimensional space. The goal of t-SNE is to take points that are "similar" in the original high-dimensional space and place them close to each other on the 2D plot. It helps us see if there are natural clusters or patterns in the data.

This is the most important takeaway from the chart. The orange dots (subscribers) and blue dots (non-subscribers) are heavily mixed together. There is no clear separation or boundary between the two classes. This visually confirms that a client who subscribes looks very similar to a client who does not across many of their features. There is no simple rule or combination of features that can cleanly separate the two groups. This means it is a difficult classification problem.

Still we can observe that some local structures are visible. There are small "islands" or "clumps" of data points. Some of these clumps have a slightly higher concentration of orange dots than others. This suggests that there are specific niches or profiles of clients that are more likely to subscribe.


In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Prepare data for t-SNE by one-hot encoding categorical features
bank_data_encoded = pd.get_dummies(bank_data.drop('Class', axis=1))
X = bank_data_encoded
y = bank_data['Class']

# Scale the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)

# Plot the t-SNE result
tsne_df = pd.DataFrame(data=X_tsne, columns=['TSNE1', 'TSNE2'])
tsne_df['Class'] = y.values

plt.figure(figsize=(10, 8))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='Class', data=tsne_df, alpha=0.7)
plt.title('t-SNE Visualization of Bank Data')
plt.show()

Next, we will apply a UMAP (Uniform Manifold Approximation and Projection) plot. Much like t-SNE, its purpose is to visualize high-dimensional data in a 2D space to reveal underlying patterns and structures.  UMAP is often considered better at preserving the global structure of the data. While t-SNE is great at showing which points are in the same local neighborhood, UMAP tries to also represent how larger clusters are related to each other. It is also generally much faster to compute.

Just like with the t-SNE plot, the most immediate takeaway is that the blue and orange dots are heavily intermingled. This again provides strong, independent confirmation that this is a difficult classification problem. There is no simple boundary that separates subscribers from non-subscribers.

Notice how UMAP creates more compact and dense "islands" of data. There seems to be more empty space between these clusters compared to the t-SNE plot. This is a characteristic of UMAP—it excels at showing that there are distinct groups (or manifolds) in the data. This suggests that there are several well-defined customer profiles or segments. For example, one island might be "young professionals with low balance," another might be "retired clients with high balance," and so on.

To identify such "islands" we use subgroup discovery techniques, which we will explore in the next step.

In [ ]:
import umap.umap_ as umap
from sklearn.preprocessing import StandardScaler

# We can reuse the scaled, one-hot encoded data from the previous step
# X_scaled is your feature matrix, y is the target variable
X = bank_data_encoded
y = bank_data['Class']

# Scale the data if it's not already scaled
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_scaled)

# Plot the UMAP result
umap_df = pd.DataFrame(data=X_umap, columns=['UMAP1', 'UMAP2'])
umap_df['Class'] = y.values

plt.figure(figsize=(10, 8))
sns.scatterplot(x='UMAP1', y='UMAP2', hue='Class', data=umap_df, alpha=0.7)
plt.title('UMAP Visualization of Bank Data')
plt.show()

Now we will apply subgroup discovery techniques to identify interesting subgroups in the bank dataset. We will use the pysubgroup library to find subgroups that are significantly different from the overall population in terms of their subscription behavior. We will start with the BeamSearch algorithm, which is a popular method for subgroup discovery. The goal is to find subgroups that have a high proportion of customers who subscribed to a term deposit compared to the overall population.

To compare the results, we will also run the Apriori algorithm, which is another method for subgroup discovery. The results from both algorithms will be compared to see if they identify similar or different subgroups.

The interpretation of the results is as follows: Each subgroup is represented by a rule that describes the conditions under which customers are more likely to subscribe to a term deposit. The quality of each subgroup is measured by the Weighted Relative Accuracy (WRAcc) score, which indicates how much better the subgroup performs compared to the overall population. The size of each subgroup is also provided, indicating how many customers belong to that subgroup.

Specifically the WRAcc score i calculated as follows:
WRAcc = (p - q) / (p + q)

where:
- p is the proportion of positive examples (subscribers) in the subgroup
- q is the proportion of positive examples in the overall population
The WRAcc score ranges from -1 to 1, where:
- 1 indicates a perfect subgroup (all members are subscribers)
- 0 indicates a subgroup that is no better than random guessing
- Negative values indicate a subgroup that is worse than random guessing (more non-subscribers than subscribers)

The "Quality" score is not the raw success rate of the subgroup. Instead, it measures how much better the subgroup is compared to this low baseline, while also considering the size of the group. Within the subgroup identified by a rule with a WRAcc of 0.04 (like duration >= 368.0), the probability of randomly selecting a subscriber is almost 50%.

In [ ]:
import pysubgroup as ps

# The task definition remains the same
positive_class = bank_data['Class'].unique()[1]
data_for_subgroup = bank_data.copy()
data_for_subgroup['target'] = (data_for_subgroup['Class'] == positive_class).astype(int)

target = ps.BinaryTarget('target', 1)
search_space = ps.create_selectors(data_for_subgroup.drop(columns=['Class', 'target']))

task = ps.SubgroupDiscoveryTask(
    data_for_subgroup,
    target,
    search_space,
    result_set_size=5,
    depth=2,
    qf=ps.WRAccQF())

# Run the search with the Apriori algorithm, disabling Numba
print("--- Apriori Algorithm Results ---")
apriori_result = ps.Apriori(use_numba=False).execute(task)

# Print the results from Apriori
result_df = apriori_result.to_dataframe()
for row in result_df.itertuples(index=False):
    print(f"Rule: {row.subgroup}, Quality: {row.quality:.2f}, Size: {row.size_sg}")

# For comparison, here are the BeamSearch results again
print("\n--- BeamSearch Algorithm Results (for comparison) ---")
beam_result = ps.BeamSearch().execute(task)
result_df_beam = beam_result.to_dataframe()
for row in result_df_beam.itertuples(index=False):
    print(f"Rule: {row.subgroup}, Quality: {row.quality:.2f}, Size: {row.size_sg}")

A Pair Plot (also called a scatterplot matrix) is a powerful tool for exploring the relationships between multiple variables at once. It's a grid of plots where:

1. The Diagonal (Top-left to bottom-right): These plots show the distribution of a single variable. In this case, they are Kernel Density Estimate (KDE) plots, which are like smoothed histograms. They show the probability density of each variable, separated by Class.
- Blue Curve: Distribution for Class 1 (Non-subscribers).
- Orange Curve: Distribution for Class 2 (Subscribers).
2. The Off-Diagonal: These are standard scatter plots that show the relationship between two different variables. For example, the plot in the top row, second column, shows duration (y-axis) vs. balance (x-axis). Each dot is a single customer.

1. The Dominance of duration (Top Row & First Column)
- Top-Left Plot (Distribution of duration): This is the single most revealing plot in the entire grid.
- The blue curve (Class 1) is a massive, sharp spike very close to zero. This means the vast majority of non-subscribers had very short phone calls.
- The orange curve (Class 2) is much flatter, more spread out, and centered further to the right. This means subscribers had significantly longer calls, on average.
- Interpretation: The two distributions are almost completely different. Knowing the call duration allows you to separate the classes with high accuracy. This is the visual proof of the "data leakage" we've discussed.
- Scatter Plots in the Top Row (duration vs. Others):
Look across the top row. In every plot (duration vs. balance, duration vs. age, duration vs. campaign), the orange dots (subscribers) are consistently higher up than the blue dots.
- Interpretation: It doesn't matter what your age, balance, or campaign history is; if your call duration is long, you are far more likely to be a subscriber. This reinforces that duration overpowers all other features.
2. The Nuanced Role of age (Third Row & Column)
- Third Plot on the Diagonal (Distribution of age):
- The blue curve (Class 1, non-subscribers) has a large, single peak around ages 30-40.
- The orange curve (Class 2, subscribers) is more interesting. It's flatter and seems to have two modes (peaks): one that mirrors the non-subscribers around 30-40, and another smaller but distinct bump for clients over 60.
- Interpretation: This visually confirms the non-linear relationship with age. While many subscribers are in the 30-40 range, there is a disproportionately high success rate among older clients, which a simple correlation would miss.
3. The Overlap in Other Features
- Scatter Plots Not Involving duration: Now, mentally cover up the top row and first column. Look at the remaining scatter plots (e.g., age vs. balance, campaign vs. age).
- In these plots, the blue and orange dots are heavily intermingled. There is no clear line, curve, or region that separates them.
- Interpretation: This is the visual proof for why the t-SNE and UMAP plots showed so much overlap. Without the duration feature, it is very difficult to separate the classes based on these other numerical features alone. This confirms that it's a hard classification problem.
- Diagonal Plots for balance and campaign: The distributions for balance and campaign are heavily skewed (most clients have low balances and were contacted few times). The blue and orange curves largely overlap, showing these features on their own are not great separators.

In [ ]:
# Select a few key numerical columns for the pair plot
pairplot_cols = ['duration', 'balance', 'age', 'campaign', 'Class']
sns.pairplot(bank_data[pairplot_cols], hue='Class', plot_kws={'alpha': 0.5})
plt.suptitle('Pair Plot of Key Numerical Features', y=1.02)
plt.show()